В качестве примера для изучения производительности Python
будем использовать умножение матриц.

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                for j in range(n_cols)])
        return M

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

Пример: функция matrix_product

In [2]:
def matrix_product(X, Y):
    """Computes the matrix product of X and Y.
    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6]])
    >>> matrix_product(X, Y)
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >>> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # верим, что с размерностями всё хорошо
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z


## Измерение времени работы

Модуль **timeit** реализует одноимённую функцию **timeit**,
которую можно использовать для измерения скорости
работы кода на Python:

In [7]:
import timeit

In [8]:
setup = """
import random
from faster_python_faster import Matrix, matrix_product
shape = 64, 64
X = Matrix.random(shape)
Y = Matrix.random(shape)
"""

In [9]:
timeit.timeit("matrix_product(X, Y)", setup, number=10)

1.7897049953942388

“Магическая” команда timeit

In [3]:
shape = 64, 64

In [4]:
X, Y = Matrix.random(shape), Matrix.random(shape)

In [10]:
%timeit matrix_product(X, Y)

211 ms ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit -n100 matrix_product(X, Y)

195 ms ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


+ Умножение двух случайных матриц из целых чисел
размера 64x64 занимает чуть меньше 200 миллисекунд.
+ 5 умножений матриц в секунду. Подозрительно медленно.
+ В чём может быть проблема?


Опередлим вспомогательную функцию bench, которая
генерирует случайные матрицы указанного размера, а затем
n_iter раз умножает их в цикле.

In [13]:
def bench(shape=(64, 64), n_iter=16):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)
        
if __name__ == "__main__":
    bench()

## Модуль cProfile

Модуль cProfile позволяет профилировать код на Python с
точностью до вызова функции или метода:

In [14]:
import cProfile

In [19]:
source = open("faster_python_faster.py").read()

In [20]:
cProfile.run(source, sort="tottime")

         41389 function calls in 3.064 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    3.034    0.190    3.035    0.190 <string>:23(matrix_product)
     8192    0.010    0.000    0.019    0.000 random.py:170(randrange)
     8192    0.007    0.000    0.009    0.000 random.py:220(_randbelow)
     8192    0.004    0.000    0.024    0.000 random.py:214(randint)
      128    0.004    0.000    0.028    0.000 <string>:14(<listcomp>)
     8213    0.002    0.000    0.002    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.001    0.001    3.064    3.064 <string>:47(bench)
       16    0.001    0.000    0.001    0.000 <string>:8(<listcomp>)
     8192    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
        1    0.001    0.001    3.064    3.064 {built-in method builtins.exec}
        2    0.000    0.000    0.028    0.014 <string>:10(random)
        1    0.000    0.000    3.0

## Модуль line_profiler

В отличие от cProfile модуль line_profiler анализирует
время работы с точностью до строки в исходном коде.

$ pip install line_profiler

Модуль расширяет систему “магических” команд IPython
командой lprun. Чтобы воспользоваться ей, сначала нужно
загрузить файл расширения:


In [21]:
%load_ext line_profiler

In [24]:
from faster_python_faster import matrix_product, bench

ImportError: cannot import name 'bench'

In [25]:
%lprun -f matrix_product bench()

Операция list.\_\_getitem__ не бесплатна. Запомним значения
X[i] и Z[i] и переставим местами циклы for так, чтобы код
делал меньше обращений по индексу.

In [26]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z


In [27]:
%lprun -f matrix_product bench()

Больше 30% времени уходит на работу внутренней машинерии
цикла for. В данном случае цикл for можно заменить на
выражение-генератор.

In [28]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z


In [29]:
%lprun -f matrix_product bench()

Почти всё время функция matrix_product проводит в самом
внутреннем цикле. Попробуем убрать из него ещё одно
обращение по индексу, транспонировав матрицу Y.

In [30]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose() # <--
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
        return Z

In [31]:
%lprun -f matrix_product bench()

AttributeError: 'Matrix' object has no attribute 'transpose'

Измерить время работы функции можно с помощью
функции timeit из модуля timeit.

Найти узкое место в программе — с помощью модуля
cProfile.

Оценить вклад каждой строки кода в общее время работы
функции — с помощью модуля line_profiler.

В IPython есть “магические” команды для всех трёх типов
измерений:
+ %timeit,
+ %prun,
+ %lprun

NumPy — библиотека для научных вычислений на Python.

В основе библиотеки — ndarray — многомерный
типизированный массив.

Сравним результаты наших стараний с ndarray:


In [32]:
import numpy as np

In [33]:
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

In [34]:
%timeit -n100 X.dot(Y)

504 µs ± 277 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


+ Дальнейшие способы ускорения кода на Python
предполагают его преобразование в машинный код либо
до, либо в момент его исполнения.

+ Ahead-of-time компиляция.
    + Python C-API: пишем код на С и реализуем к нему интерфейс, понятный интерпретатору CPython.
    + Пишем код на подмножестве Python и преобразуем его в код на C++ (**Pythran**) или C (**Cython**), использующий C-API интепретатора CPython.
+ Just-in-time компиляция: пишем код на Python и пытаемся
сделать его быстрее в момент исполнения.
    + **PyPy**: следим за исполнением программы и компилируем в машинный код наиболее частые пути в ней.
    + Транслируем специальным образом помеченный код на Python в LLVM (**Numba**) или C++ (**HOPE**), а затем компилируем в машинный код.


# Numba

Поставить Numba с помощью pip может быть непросто.
Рекомендуемый способ установки описан по ссылке:
http://bit.ly/install-numba.

На момент версии 0.21 Numba не умеет эффективно
оптимизировать код со списками, поэтому нам придётся
переписать функцию matrix_product с использованием
ndarray:


In [35]:
import numba

Для использования Numba достаточно декорировать
функцию с помощью numba.jit.

В момент первого вызова функция будет транслирована в
LLVM и скомпилирована в машинный код:


In [36]:
@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [37]:
%timeit -n100 jit_matrix_product(X, Y)

The slowest run took 38.10 times longer than the fastest. This could mean that an intermediate result is being cached.
2.52 ms ± 5.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Декоратор *numba.jit* пытается вывести типы аргументов и
возвращаемого значения декорируемой функции:

!!!Важный момент!!! 

Каждый раз, когда в массивах numpy мы пишем X[i][j] - это создает слайс.

Лучшим решением является обращение по объедененному индексу X[i, j]

Что бы код, написанные с использованеием *numba* работал, его нужно писать так, как он бы писался на низкоуровневом языке.

Например, если код содержит вызовы Python функций, то
ускорение от компиляции кода может быть
незначительным:

In [38]:
@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for k in range(n_ycols):
            Z[i, k] = (X[i, :] * Y[:, k]).sum()
    return Z


In [39]:
%timeit -n100 jit_matrix_product(X, Y)

The slowest run took 6.63 times longer than the fastest. This could mean that an intermediate result is being cached.
2.44 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Numba — это JIT компилятор для Python кода, основанный
на трансляции в LLVM.

В теории Numba не требует никаких изменений в коде,
кроме использования декоратора numba.jit.

На практике далеко не любой код поддаётся эффективной
оптимизации.

Еще имеется:
+ явная аннотация типов,
+ интеграция с CUDA,
+ AOT компиляция кода, использующего Numba.

Почитать об этом можно в документе http://numba.pydata.org.

# Cython

Cython — это:
+ типизированное расширение(любой код на Python — это корректный код на Cython)
языка Python,
+ оптимизирующий компилятор Python и Cython в код на C,
использующий C-API интерпретатора CPython.


In [40]:
%load_ext cython

In [41]:
%%cython
print("Hello, world!")

Hello, world!


Узнать подробности о взаимодействии Cython с системой
импортов и библиотекой setuptools можно на сайте
проекта: http://bit.ly/compile-cython.

“Магическая” команда cython компилирует содержимое ячейки
с помощью Cython, а затем загружает все имена из
скомпилированного модуля в глобальное пространство имён.

In [42]:
%%cython
def f():
    return 42

def g():
    return []

In [43]:
f

<function _cython_magic_25305ca53620d46b4ccb15d1d251c5f9.f>

In [44]:
g

<function _cython_magic_25305ca53620d46b4ccb15d1d251c5f9.g>

In [45]:
f(), g()

(42, [])

In [46]:
%%cython
from faster_python_faster import Matrix

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [47]:
%timeit -n100 cy_matrix_product(X, Y)

102 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Компилятор Cython поддерживает опциональную
типизацию.

In [49]:
%%cython -a
from faster_python_faster import Matrix

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

Чем интенсивнее цвет, тем менее специфичен тип
выражения и тем медленней выполняется фрагмент кода.

Обилие желтого цвета намекает, что результат компиляции
функции cy_matrix_product мало чем отличается от её
версии на Python, потому что все объекты имеют тип
PyObject.


К сожалению, списки в Python гетерогенны, поэтому, как и в
случае с Numba, мы вынуждены перейти к использованию
ndarray:

In [50]:
%%cython -a
import numpy as np
def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [51]:
%timeit -n100 cy_matrix_product(X, Y)

172 ms ± 8.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Cython и типизация

In [52]:
%%cython -a
import numpy as np # Это работает как импорт
cimport numpy as np # Это работает как include и не перезаписывает прошлый импорт

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [53]:
%timeit -n100 cy_matrix_product(X, Y)

183 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

Несмотря на то что все переменные имеют явный тип, тип
элемента ndarray всё ещё не определён, поэтому тело самого
вложенного цикла ярко-жёлтое.

In [55]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y):
    
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
    np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z


In [59]:
%timeit -n100 cy_matrix_product(X, Y)

ValueError: Buffer dtype mismatch, expected 'int64_t' but got 'long'

Cython позволяет пожертвовать безопасностью в пользу
производительности, отключив проверки выхода за границы
массива и проверки переполнения в целочисленных
операциях.

In [57]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False) # директива, отменяющая проверку массива
@cython.overflowcheck(False) # директива, отменяющая проверку на переполнение
def cy_matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y):
    
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
    np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z


In [60]:
%timeit -n100 cy_matrix_product(X, Y)

ValueError: Buffer dtype mismatch, expected 'int64_t' but got 'long'

Cython — удобный инструмент для написания критичного
по производительности кода на Python-подобном
синтаксисе.

Еще есть:
+ C-функций и типы расширения,
+ взаимодействия Cython с кодом на C и C++*,
+ многопоточность,
+ профилирование и отладка.

Обо всём этом и многом другом можно узнать из
документации: http://docs.cython.org.
